# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Необходимо построить модель с максимально большим значением *accuracy*. Доля правильных ответов должна быть больше 0.75.

## Загрузка и анализ данных

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
display(df.head())
display(df.info())
display(df.isna().sum())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

## Разделим данные на выборки

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Разобьем данные на обучающие - train - 60%, валидационные - valid 20% и тестовые - test 20%

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

In [ ]:
print('Shape выборок:')
print('features_train', features_train.shape)
print('target_train', target_train.shape)
print('')
print('features_valid', features_valid.shape)
print('target_valid', target_valid.shape)
print('')
print('features_test', features_test.shape)
print('target_test', target_test.shape)

Shape выборок:
features_train (1928, 4)
target_train (1928,)

features_valid (643, 4)
target_valid (643,)

features_test (643, 4)
target_test (643,)


**Вывод**

В качестве параметров мы взяли все столбцы кроме `is_ultra`

Столбец `is_ultra` - целевой признак

Мы извлекли признаки для изучения в `features`, а целевой признак в `target`

Далее выделили 20% для валидационной выборки и 20% для тестовой выборки, остальные для обучающей

## Исследуем модели

Напишем функцию для подбора параметров

In [ ]:
def get_best_estimator(model, params, _scoring, features, target):
    GSCV = GridSearchCV(model, params,
                        cv = 3, # количество разбиений на кросс-валидацию
                        scoring = _scoring
                       )
    GSCV.fit(features, target)
    print(GSCV.best_estimator_)
    return GSCV.best_estimator_

### Найдем лучшее дерево решения

In [ ]:
best_model_dtc = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model_dtc = model
        best_result = result
        best_depth = depth

print("Accuracy лучшей модели:", best_result, "Глубина дерева:", best_depth)

Accuracy лучшей модели: 0.7853810264385692 Глубина дерева: 3


In [ ]:
best_model_dtc.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [ ]:
params = {
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "max_features" : [1,2,3,4],
    "min_samples_leaf": [5, 10, 15],
    "random_state" : [12345]
}

best_model_dtc = get_best_estimator(best_model_dtc, params, 'accuracy', features_train, target_train)

DecisionTreeClassifier(max_depth=6, max_features=3, min_samples_leaf=5,
                       random_state=12345)


In [ ]:
best_model_dtc.fit(features_train, target_train)
predictions_valid = best_model_dtc.predict(features_valid)
accuracy_score(target_valid, predictions_valid)

0.7853810264385692

### Найдем лучшую модель для случайных лесов

In [ ]:
best_model_rmc = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model_rmc = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 8


In [ ]:
best_model_rmc.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [ ]:
params = {
    "max_depth": [5, 6, 7, 8, 9, 10],
    "max_features" : [1,2,3,4],
    #"min_samples_leaf" : [5, 10, 15, 20, 25, 30],
    "min_samples_split" : [5, 10, 15, 20, 25, 30],
    "n_estimators" : [1, 5, 10, 20, 30, 40, 50],
    "random_state" : [12345]
    #"oob_score" : [True, False]


}

best_model_rmc = get_best_estimator(best_model_rmc, params, 'accuracy', features_train, target_train)

RandomForestClassifier(max_depth=9, max_features=3, min_samples_split=20,
                       n_estimators=10, random_state=12345)


In [ ]:
best_model_rmc.score(features_valid, target_valid)

0.7916018662519441

### Найдем лучшую модель логистической регресии

In [ ]:
best_model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

In [ ]:
best_model_lr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [ ]:
params = {
    "max_iter": [100, 500, 1000, 1500],
    "solver" : ['lbfgs'],
    "random_state" : [12345]
}

best_model_lr = get_best_estimator(best_model_lr, params, 'accuracy', features_train, target_train)

LogisticRegression(random_state=12345)


In [ ]:
#predictions = best_model_lr.predict(features_valid)
best_model_lr.score(features_valid, target_valid)

0.7107309486780715

**Вывод**

Мы построили модели с различными гиперпараметрами используя два типа моделей: `Дерево решений`, `Случайный лес` и `Логистическая регрессия`

Для `Дерево решений` лучшее значение параметра `max_depth` = 3. `Accuracy` = 0.7853810264385692

Для модели `Случайный лес` параметры `Количество деревьев` = 40, `Максимальная глубина` = 8. `Accuracy` = 0.8087091757387247

Модель логистической регрессии имеет `Accuracy` = 0.7107309486780715

## Проверим модели на тестовой выборке

### Проверим лучшую модель "Дерево решений" на тестовой выборке

In [ ]:
predictions_test = best_model_dtc.predict(features_test)
result_test = accuracy_score(target_test, predictions_test)
result_test

0.7869362363919129

### Проверим лучшую модель "Случайный лес" на тестовой выборке

In [ ]:
result_test = best_model_rmc.score(features_test, target_test)
result_test

0.8055987558320373

### Проверим лучшую модель "Логистическая регрессия" на тестовой выборке

In [ ]:
result_test = best_model_lr.score(features_test, target_test)
result_test

0.6842923794712286

### Вывод

Мы протестировали все модели на тестовых данных и получили следующие результаты:
    
- `Дерево решений`:
    - max_depth=6,
    - max_features=3,
    - min_samples_leaf=5,
    - accuracy = 0.7869362363919129
- `Случайный лес`:
    - max_depth=9,
    - max_features=3,
    - min_samples_split=20,
    - n_estimators=10,
    - accuracy = 0.8055987558320373
- `Логистическая регрессия`:
    - accuracy = 0.6842923794712286
    
Таким образом мы нашли лучшую модель с точностью более 0.75:
- `Случайный лес`:
    - max_depth=9,
    - max_features=3,
    - min_samples_split=20,
    - n_estimators=10,
    - accuracy = 0.8055987558320373

## Проверим модели на адекватность

Посмотрим на долю наибольшего класса.

In [ ]:
df['is_ultra'].mean()

0.30647168637212197

Только 30% относятся к тарифам Ultra. Наши модели адекватны.
